<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [DCGAN01] - A first DCGAN to Draw a Sheep
<!-- DESC --> Episode 1 : Draw me a sheep, revisited with a DCGAN
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Build and train a DCGAN model with the Quick Draw dataset
 - Understanding DCGAN

The [Quick draw dataset](https://quickdraw.withgoogle.com/data) contains about 50.000.000 drawings, made by real people...  
We are using a subset of 117.555 of Sheep drawings  
To get the dataset : [https://github.com/googlecreativelab/quickdraw-dataset](https://github.com/googlecreativelab/quickdraw-dataset)  
To get the subdataset in numpy bitmap file : [https://console.cloud.google.com/storage/quickdraw_dataset/full/numpy_bitmap](https://console.cloud.google.com/storage/quickdraw_dataset/full/numpy_bitmap) (94.3 Mo)


## What we're going to do :

 - Have a look to the dataset
 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard

## Acknowledgements :
Thanks to **François Chollet** who is at the base of this example.  
See : [https://keras.io/examples/](https://keras.io/examples/)


## Step 1 - Init python stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard

from modules.models    import DCGAN
from modules.callbacks import ImagesCallback

sys.path.append('..')
import fidle.pwk as pwk

run_dir = './run/DCGAN.001'                  # Output directory
datasets_dir = pwk.init('DCGAN01', run_dir)

## Step 2 - Parameters

In [ ]:
latent_dim = 128

## Step 3 - Load dataset and have a look 
Load sheeps as numpy bitmaps...

In [ ]:
x_data = np.load(datasets_dir+'/QuickDraw/origine/full_numpy_bitmap_sheep.npy')
x_data = x_data/255
x_data=x_data.reshape(-1,28,28,1)
x_data.shape

...and have a look :  
Note : These sheep are sheep drawn ... by real humans!

In [ ]:
pwk.plot_images(x_data.reshape(-1,28,28), indices=range(72), columns=12, x_size=1, y_size=1, y_padding=0,spines_alpha=0, save_as='01-Sheeps')

## Step 4 - Create a discriminator

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

## Step 5 - Create a generator

In [ ]:
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(7 * 7 * 64),
        layers.Reshape((7, 7, 64)),
        layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()


## Step 6 - Create our DCGAN !

In [ ]:
gan = DCGAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

In [ ]:
gan.compile(
    discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    generator_optimizer     = keras.optimizers.Adam(learning_rate=0.0001),
    loss_function           = keras.losses.BinaryCrossentropy(),
)

In [ ]:
gan.fit( x_data, epochs=1, batch_size=32, callbacks=[ImagesCallback(num_img=10, latent_dim=latent_dim)]
)

In [ ]:
a=np.array([[1,2,3], [1,2,3],])
a.shape